In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import kaggle
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0




  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|##########| 200k/200k [00:00<00:00, 658kB/s]
100%|##########| 200k/200k [00:00<00:00, 658kB/s]


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip') 
zip_ref.extractall() # extract file to directory
zip_ref.close() # close file

### We read the **csv** file as a dataframe using Pandas.

In [5]:
df = pd.read_csv('orders.csv',na_values=['Not Available','unknown'])
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [6]:
# Explore the columns to see some of them need cleaning/dropping/feature engineering.

df.columns

Index(['Order Id', 'Order Date', 'Ship Mode', 'Segment', 'Country', 'City',
       'State', 'Postal Code', 'Region', 'Category', 'Sub Category',
       'Product Id', 'cost price', 'List Price', 'Quantity',
       'Discount Percent'],
      dtype='object')

#### Most of the column names have capital letters in the beginning and sometimes have spaces, which need to be replaced by underscores.

In [8]:
df.columns = df.columns.str.lower()

In [9]:
df.columns = df.columns.str.replace(' ', '_')

In [10]:
df.head(2)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3


#### We create another column ***discount_amount*** that expresses the discount amount not in percentages but in actual value.

In [11]:
df['discount_amount'] = df['list_price'] * df['discount_percent'] * 0.01
df['sold_price'] = df['list_price'] - df['discount_amount'] # This column is to calculate the price of the profuct after considering discount

In [12]:
df['profit'] = df['sold_price'] - df['cost_price'] # Calculates the profit = Sold price - Cost price
df

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount_amount,sold_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,2023-02-18,Second Class,Consumer,United States,Miami,Florida,33180,South,Furniture,Furnishings,FUR-FU-10001889,30,30,3,4,1.2,28.8,-1.2
9990,9991,2023-03-17,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Furniture,Furnishings,FUR-FU-10000747,70,90,2,4,3.6,86.4,16.4
9991,9992,2022-08-07,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Technology,Phones,TEC-PH-10003645,220,260,2,2,5.2,254.8,34.8
9992,9993,2022-11-19,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Office Supplies,Paper,OFF-PA-10004041,30,30,4,3,0.9,29.1,-0.9


#### Now let's verify if all the features have the appropriate data types. 

In [13]:
df.dtypes

order_id              int64
order_date           object
ship_mode            object
segment              object
country              object
city                 object
state                object
postal_code           int64
region               object
category             object
sub_category         object
product_id           object
cost_price            int64
list_price            int64
quantity              int64
discount_percent      int64
discount_amount     float64
sold_price          float64
profit              float64
dtype: object

#### We can see that the ***order_date*** features is currently an object-type feature. We can, and should, convert it into a date_time format.

In [14]:
df['order_date'] = pd.to_datetime(df['order_date'], format="%Y-%m-%d") 

In [15]:
df = df.drop(columns=['list_price', 'cost_price', 'discount_percent']) # These columns aren't importannt anymore

In [16]:
df.head(3)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount_amount,sold_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,0.5,9.5,-0.5


#### SQLAlchemy Database 
##### Now we establish a connection to a Microsoft SQL Server database using SQLAlchemy and PyMySQL.

In [39]:
import sqlalchemy as sal
import pymysql

engine = sal.create_engine('mssql://Pirouette/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn = engine.connect()

In [40]:
df.to_sql('df_order', con=conn, index=False, if_exists = 'replace')

# In SQL, the table name will be df_order
# con = conn wil establish the connection (conn) from where the data will be loaded.
# Index = False will delete the addtional index column from the dataframe.
# if_exists = 'replace', creates the df_order table automatically

38

##### But upon a slight inspection into the columns properties in the Microsoft SQL Server Management, we see that due to the ***if_exists = 'replace'*** option, pandas creates a table with the highest possible data types. The way out of this problem is to create the table yourself. And instead of ***replace***, use ***append***.

In [41]:
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'quantity', 'discount_amount', 'sold_price', 'profit'],
      dtype='object')

In [43]:
df.to_sql('df_orders', con=conn, index=False, if_exists = 'append')

38